---
title: Block Krylov Iteration
description: Advanced techniques for improved accuracy and robustness in randomized low-rank approximation
keywords: [subspace iteration, block Krylov, power iteration, Krylov subspace, iterative methods, spectral gap, convergence acceleration]
numbering:
  equation:
    enumerator: 5.%s
    continue: true
  proof:theorem:
    enumerator: 5.%s
    continue: true
  proof:algorithm:
    enumerator: 5.%s
    continue: true
  proof:definition:
    enumerator: 5.%s
    continue: true
  proof:proposition:
    enumerator: 5.%s
    continue: true
---



## Motivation 

[Subspace Iteration](./subspace-iteration.ipynb) can dramatically improve on the performance of the [Randomized SVD](./randomized-svd.ipynb) when $\vec{A}$ has a heavy tail. 
The key observation is that the singular value tail of $(\vec{A}\vec{A}^\T)^q\vec{A}$ is much lighter than that of $\vec{A}$.
This is because the polynomial $x^{2q+1}$ pushes tail singular values of $\vec{A}$ to zero.

From a computational perspective, there isn't that much special about $x^{2q+1}$. In fact, we can replace this monomial with any polynomial of the same degree containing only odd degree terms, without significantly changing the cost of the algorithm.
Thus, we might consider an approximation of the form:
\begin{equation}
\widehat{\vec{A}} = \vec{Q}\vec{Q}^\T \vec{A},
\quad 
\vec{Q} = \Call{orth}(p(\vec{A}\vec{A}^\T)\vec{A}\vec{\Omega})
,\quad \deg(p) \leq q.
\end{equation}
It's conceivable that, if we choose a good polynomial, this method could be more accurate than the subspace iteration.[^cheb]
The difficulty is that we don't know how to choose a good polynomial in advance, since we don't know the singular values of $\vec{A}$.

[^cheb]: For instance, Chebyshev polynomials grow substantially faster than monomials.


## Block Krylov Iteration

Observe that 
\begin{equation}
\{ p(\vec{A}\vec{A}^\T)\vec{A}\vec{\Omega} : \deg(p) \leq q \}
= \mathcal{K}_t(\vec{A}\vec{A}^\T,\vec{A}\vec{\Omega}),
\end{equation}
where $\mathcal{K}_t(\vec{A}\vec{A}^\T,\vec{A}\vec{\Omega})$ is the block Krylov subspace
\begin{equation}
\mathcal{K}_t(\vec{A}\vec{A}^\T,\vec{A}\vec{\Omega}) := \operatorname{span}\left\{ \vec{A}\vec{\Omega}, (\vec{A}\vec{A}^\T)\vec{A}\vec{\Omega}, \ldots, (\vec{A}\vec{A}^\T)^{t-1}\vec{A}\vec{\Omega} \right\}.
\end{equation}

Computing a basis for $\mathcal{K}_t(\vec{A}\vec{A}^\T,\vec{A}\vec{\Omega})$ requires the same number of matrix-vector products with $\vec{A}$ and $\vec{A}^\T$ as subspace iteration.
This suggests the *Randomized Block Krylov Iteration* (RBKI) approximation:
\begin{equation}
\widehat{\vec{A}} = \vec{Q}\vec{Q}^\T \vec{A},
\quad 
\vec{Q} = \Call{orth}(\mathcal{K}_t(\vec{A}\vec{A}^\T,\vec{A}\vec{\Omega})).
\end{equation}
Intuitively, by projecting onto a larger space, we can get a better approxiamtion.

## Convergence Guarantees



RBKI with $\sqrt{q}$ iterations can achieve similar guarantees to subspace iteration with $q$ iterations.
This was first proved in {cite:p}`musco_musco_15`; see also {cite:p}`tropp_webber_23` for a non-asymptotic analysis.


### Block-size versus depth

{cite:p}`chen_epperly_meyer_musco_rao_25`

{cite:p}`meyer_musco_musco_24`


:::{prf:theorem} 
Let $\widehat{\vec{A}}$ be the rank-$k$ approximation to $\vec{A}$ produced by RBKI after $t$ iterations. 
Then for some
\begin{equation*}
t = \tilde{O}\left( \frac{k/b}{\sqrt{\varepsilon}}  \right).
\end{equation*}
it holds that
\begin{equation*}
\|\vec{A} - \llbracket \widehat{\vec{A}}\rrbracket_k \| \leq (1+\varepsilon) \|\vec{A} - \llbracket \vec{A} \rrbracket_k\|.
\end{equation*}
::: 

Non-asymptotic bounds for RSVD, RSI, and RBKI can be found in {cite:p}`tropp_webber_23`

## Numerical Experiment

In [7]:
import numpy as np
import scipy as sp
import time
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
from randnla import *

In [ ]:
# load experiment

n = 4000
name = 'intro'
Λ = np.geomspace(1,100,n)
blocksizes = [1,5,20,100,200]
qs = [650,200,60,11,7]
skips = [20,4,1,1,1]
k = 100

A = np.diag(Λ)
err_opt = np.linalg.norm(np.sort(Λ)[:-k])

class AAt:
    def __matmul__(self,X):
        return A@(A.T@X)
        
AAt = AAt()

# run experiment
all_times = {}
all_errs = {}
for i,(b,q,skip) in enumerate(zip(blocksizes,qs,skips)):

    B = np.random.randn(n,b)
    
    Q,_,times = block_lanczos(AAt,B,q,reorth=True)

    all_times[b] = times

    all_errs[b] = np.zeros(q+1)
        
    for j in range(0,q+1,skip):

        Zj = Q[:,:b*j]

        X = Zj.T@A
        Qtilde,s,Vt = np.linalg.svd(X,full_matrices=False)

            
        A_hat = Zj@(Qtilde[:,:k]@(np.diag(s[:k])@Vt[:k]))
        err = np.linalg.norm(A - A_hat)
        
        all_errs[b][j] = err

# plot the error
fig, axs = plt.subplots(1,2,figsize=(8, 4),sharey=True)
plt.subplots_adjust(wspace=0.1)

for i,(b,q,skip) in enumerate(zip(blocksizes,qs,skips)):

    errs = all_errs[b]
    times = all_times[b]
    
    ax = axs[0]

    ax.set_ylabel(r'accuracy $\varepsilon$')

    ax.plot(np.arange(0,q+1)[::skip]*b,errs[::skip]/err_opt-1,ms=5,label=f'$b={b}$')
    ax.set_yscale('log')
    ax.set_ylim(1e-6,1e0)
    ax.set_xlabel('matrix-vector products')
      
    ax = axs[1]
    
    ax.plot(times[::skip],errs[::skip]/err_opt-1,ms=5,label=f'$b={b}$')
    ax.set_xlabel('wall-clock time (s)')
    ax.set_xlim(-.1,3)
    ax.legend()

plt.savefig(f'RBKI.svg')
plt.close()


We plot the value of $\varepsilon$ so that $\|\vec{A} -\llbracket \widehat{\vec{A}}\rrbracket_k  \|_\F \leq (1+\varepsilon) \|\vec{A} - \llbracket \vec{A} \rrbracket_k\|_\F$.


![](RBKI.svg)